In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
df = "D:\Traffic Signal Recognition\Dataset" 
ldf = 'D:\Traffic Signal Recognition\labels.csv' 
batch_size_val=32 
epochs_val=10
imageDimesions = (32,32,3)
testRatio = 0.2    
validationRatio = 0.2 

In [4]:
count = 0
images = []
classNo = []
myList = os.listdir(df)
print("Total Classes Detected:",len(myList))
noOfClasses=len(myList)

Total Classes Detected: 43


In [5]:
for x in range (0,len(myList)):
    myPicList = os.listdir(df+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(df+"/"+str(count)+"/"+y)
        images.append(curImg)
        classNo.append(count)
    print(count, end =" ")
    count +=1
print(" ")
images = np.array(images)
classNo = np.array(classNo)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42  


In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)

In [8]:
labels = pd.read_csv("D:\Traffic Signal Recognition\labels.csv")

In [9]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img

In [10]:
def equalize(img):
    img =cv2.equalizeHist(img)
    return img

In [11]:
def preprocessing(img):
    img = grayscale(img)     
    img = equalize(img)      
    img = img/255            
    return img

In [12]:
X_train=np.array(list(map(preprocessing,X_train)))  
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))

In [13]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [14]:
dataGen= ImageDataGenerator(width_shift_range=0.1,   
                            height_shift_range=0.1,
                            zoom_range=0.2,  
                            shear_range=0.1,  
                            rotation_range=10)  

In [15]:
dataGen.fit(X_train)
batches= dataGen.flow(X_train,y_train,batch_size=20)
X_batch,y_batch = next(batches)

In [16]:
y_train = to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [17]:
def myModel():
    model= Sequential()
    model.add((Conv2D(60,(5,5),input_shape=(imageDimesions[0],imageDimesions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    model.add((Conv2D(60, (5,5), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    model.add((Conv2D(30, (3,3),activation='relu')))
    model.add((Conv2D(30, (3,3), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(500,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses,activation='softmax')) 
    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
 
model = myModel()

C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
history=model.fit_generator(dataGen.flow(X_train,y_train,batch_size=32),steps_per_epoch=len(X_train)//32,epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)


C:\Users\acer\AppData\Local\Temp\ipykernel_23080\676064730.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(dataGen.flow(X_train,y_train,batch_size=32),steps_per_epoch=len(X_train)//32,epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)


Epoch 1/10
695/695 [==============================] - 102s 146ms/step - loss: 2.5104 - accuracy: 0.3029 - val_loss: 0.7685 - val_accuracy: 0.7660
Epoch 2/10
695/695 [==============================] - 121s 174ms/step - loss: 1.2025 - accuracy: 0.6350 - val_loss: 0.3203 - val_accuracy: 0.9170
Epoch 3/10
695/695 [==============================] - 114s 165ms/step - loss: 0.8447 - accuracy: 0.7427 - val_loss: 0.1808 - val_accuracy: 0.9497
Epoch 4/10
695/695 [==============================] - 96s 138ms/step - loss: 0.6530 - accuracy: 0.7956 - val_loss: 0.1233 - val_accuracy: 0.9673
Epoch 5/10
695/695 [==============================] - 96s 138ms/step - loss: 0.5461 - accuracy: 0.8284 - val_loss: 0.0885 - val_accuracy: 0.9736
Epoch 6/10
695/695 [==============================] - 105s 151ms/step - loss: 0.4779 - accuracy: 0.8536 - val_loss: 0.0818 - val_accuracy: 0.9779
Epoch 7/10
695/695 [==============================] - 96s 138ms/step - loss: 0.4136 - accuracy: 0.8692 - val_loss: 0.0703 - va

In [19]:
score =model.evaluate(X_test,y_test,verbose=0)
print('Test Score:',score[0])
print('Test Accuracy:',score[1])
 
model.save("model.h5")

Test Score: 0.04838519170880318
Test Accuracy: 0.9863505959510803
